In [ ]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import scipy

# Import data
folder = "/nfs/nas22/fs2202/biol_micro_bioinf_nccr/hardt/nguyenb/tnseq/scratch/10_22/03-23-unenriched_analysis/"
library_9_1 = pd.read_csv(folder + "library_9_1_rra_results.csv")
library_10_1 = pd.read_csv(folder + "library_10_1_rra_results.csv")
library_10_2 = pd.read_csv(folder + "library_10_2_rra_results.csv")
library_11_2 = pd.read_csv(folder + "library_11_2_rra_results.csv")
library_12_1 = pd.read_csv(folder + "library_12_1_rra_results.csv")
library_12_2 = pd.read_csv(folder + "library_12_2_rra_results.csv")
library_13_1 = pd.read_csv(folder + "library_13_1_rra_results.csv")
library_13_2 = pd.read_csv(folder + "library_13_2_rra_results.csv")
library_14_2 = pd.read_csv(folder + "library_14_2_rra_results.csv")
library_15_1 = pd.read_csv(folder + "library_15_1_rra_results.csv")

library_9_1.name = "library_9_1"
library_10_1.name = "library_10_1"
library_10_2.name = "library_10_2"
library_11_2.name = "library_11_2"
library_12_1.name = "library_12_1"
library_12_2.name = "library_12_2"
library_13_1.name = "library_13_1"
library_13_2.name = "library_13_2"
library_14_2.name = "library_14_2"
library_15_1.name = "library_15_1"

dfList = [library_9_1, library_10_1, library_10_2, library_11_2, library_12_1, library_12_2, library_13_1, library_13_2, library_14_2, library_15_1]
[df.head() for df in dfList]


They are all "inoculum", no "unenriched_inoculum" present


# Any outliers or anything that does not look right?

In [ ]:
LFCs = pd.DataFrame({"library_9_1": library_9_1["LFC"], "library_10_1": library_10_1["LFC"], "library_10_2": library_10_2["LFC"], "library_11_2": library_11_2["LFC"], "library_12_1": library_12_1["LFC"], "library_12_2": library_12_2["LFC"], "library_13_1": library_13_1["LFC"], "library_13_2": library_13_2["LFC"], "library_14_2": library_14_2["LFC"], "library_15_1": library_15_1["LFC"]})

LFCs
#[sns.stripplot(df["LFC"]) for df in dfList]
plot = sns.stripplot(LFCs)
plot.set_xticklabels(plot.get_xticklabels(), rotation=80)
plot


In [ ]:
plot = sns.violinplot(LFCs)
plot.set_xticklabels(plot.get_xticklabels(), rotation=80)
plot

In [ ]:
dfList[1].describe()


In [ ]:
dfList[2].describe()


In [ ]:
dfList[3].describe()


library_10_2 has LFCs that are more negative compared to the other libraries.

## Outliers

In [ ]:
library_11_2[library_11_2["LFC"] > 3]

In [ ]:
library_13_2[library_13_2["LFC"] < -4]

In [ ]:
library_10_2[library_10_2["LFC"] < -4]




# How many genes have fitness defect in each of the libraries? We are not expecting a lot of hits.
There are quite many genes with fitness defects.

In [ ]:
print("Name\t\t#Genes\t#Genes with |LFC|>1\tPercentage")
LFClist = [df[abs(df["LFC"])>1] for df in dfList]

[print(df.name,"\t", len(df), "\t", len(df[abs(df["LFC"])>1]), "\t\t\t", len(df[abs(df["LFC"])>1])/len(df)) for df in dfList]


In [ ]:
print("Name\t\t#Genes\t#Genes with any FDR < 0.01\tPercentage")
FDRlist = [df[(pd.DataFrame(df[["pos_selection_fdr", "neg_selection_fdr"]]<0.01).any(axis=1))] for df in dfList]

[print(df.name,"\t", len(df), "\t", len(df[(pd.DataFrame(df[["pos_selection_fdr", "neg_selection_fdr"]]<0.01).any(axis=1))]), "\t\t\t\t", len(df[(pd.DataFrame(df[["pos_selection_fdr", "neg_selection_fdr"]]<0.01).any(axis=1))])/len(df)) for df in dfList]

In [ ]:
print("Name\t\t#Genes\t#Genes with any FDR < 0.01 & |LFC|>1\tPercentage")

#dfList[1][(pd.DataFrame(pd.DataFrame(dfList[1][["pos_selection_fdr", "neg_selection_fdr"]]<0.01), dfList[1][abs(dfList[1]["LFC"])>1]).any(axis=1))]
#[print(df.name,"\t", len(df), "\t", len(df[(pd.DataFrame(df[["pos_selection_fdr", "neg_selection_fdr"]]<0.01).any(axis=1))]), "\t\t\t\t\t", len(df[(pd.DataFrame(df[["pos_selection_fdr", "neg_selection_fdr"]]<0.01).any(axis=1))])/len(df)) for df in dfList]

[print(dfList[df].name, "\t", len(FDRlist[df]), "\t", len(pd.merge(FDRlist[df], LFClist[df], on="Name", how="inner")), "\t\t\t\t", len(pd.merge(FDRlist[df], LFClist[df], on="Name", how="inner"))/len(dfList[df])) for df in range(0,len(dfList))]


# Do LFC correlate well from library to library?  (expecting 60% overlap)
There are only 90 genes in common.

In [ ]:
#LFCdf = pd.DataFrame(np.array([library_9_1["LFC"], library_10_1["LFC"], library_10_2["LFC"], library_11_2["LFC"], library_12_1["LFC"], library_12_2["LFC"], library_13_1["LFC"], library_13_2["LFC"], library_14_2["LFC"], library_15_1["LFC"]]))
# INNER JOIN
merged1 = pd.merge(library_9_1, library_10_1, on = "Name", how = "inner", suffixes =("_9_1", "_10_1"))
merged2 = pd.merge(library_10_2, library_11_2, on = "Name", how = "inner", suffixes =("_10_2", "_11_2"))
merged3 = pd.merge(library_12_1, library_12_2, on = "Name", how = "inner", suffixes =("_12_1", "_12_2"))
merged4 = pd.merge(library_13_1, library_13_2, on = "Name", how = "inner", suffixes =("_13_1", "_13_2"))
merged5 = pd.merge(library_14_2, library_15_1, on = "Name", how = "inner", suffixes =("_14_2", "_15_1"))

merged12 = pd.merge(merged1, merged2, on = "Name", how = "inner")
merged34 = pd.merge(merged3, merged4, on = "Name", how = "inner")

merged1234 = pd.merge(merged12, merged34, on = "Name", how = "inner")

merged = pd.merge(merged5, merged1234, on = "Name", how = "inner")

merged
#np.corrcoef(library_9_1["LFC"], library_10_1["LFC"])

In [ ]:
LFCdf = pd.DataFrame(np.array([merged["LFC_9_1"], merged["LFC_10_1"], merged["LFC_10_2"], merged["LFC_11_2"], merged["LFC_12_1"], merged["LFC_12_2"], merged["LFC_13_1"], merged["LFC_13_2"], merged["LFC_14_2"], merged["LFC_15_1"]]))
LFCdf_transposed = LFCdf.T
LFCdf_transposed.columns = ["LFC_9_1", "LFC_10_1", "LFC_10_2", "LFC_11_2", "LFC_12_1", "LFC_12_2", "LFC_13_1", "LFC_13_2", "LFC_14_2", "LFC_15_1"]
#sns.color_palette("icefire", as_cmap=True)
sns.heatmap(LFCdf_transposed.corr("spearman"), cmap = "icefire") # non parametric for larger datasets (90 rows)

#### Focus on pairwise correlation

In [ ]:
print(" LFC_9_1 and LFC_10_1:", "\t\t","genes in common:", len(merged1) ,"| Spearman rho: ", scipy.stats.spearmanr(merged1["LFC_9_1"], merged1["LFC_10_1"]))
print(" LFC_10_2 and LFC_11_2:", "\t", "genes in common:", len(merged2) ,"| Spearman rho: ", scipy.stats.spearmanr(merged2["LFC_10_2"], merged2["LFC_11_2"]))
merged6 = pd.merge(library_10_2, library_15_1, on = "Name", how = "inner", suffixes =("_10_2", "_15_1"))
print(" LFC_10_2 and LFC_15_1:", "\t", "genes in common:", len(merged6) ,"| Spearman rho: ", scipy.stats.spearmanr(merged6["LFC_10_2"], merged6["LFC_15_1"]))
print(" LFC_12_1 and LFC_12_2:", "\t", "genes in common:", len(merged3) ,"| Spearman rho: ", scipy.stats.spearmanr(merged3["LFC_12_1"], merged3["LFC_12_2"]))
print(" LFC_13_1 and LFC_13_2:", "\t", "genes in common:", len(merged4) ,"| Spearman rho: ", scipy.stats.spearmanr(merged4["LFC_13_1"], merged4["LFC_13_2"]))
print(" LFC_14_2 and LFC_15_1:", "\t", "genes in common:", len(merged5) ,"| Spearman rho: ", scipy.stats.spearmanr(merged5["LFC_14_2"], merged5["LFC_15_1"]))


# Are there genes that have defects in more than 1 library? If so, in how many libraries?
## What is the biological function of the genes that have fitness defect? Does it make sense that these would be lost during enrichment (we are expecting genes with growth defects in LB to show up). 

90 genes are present in all dataframes

9 genes have defects in all libraries: pilR, pilV, traY, yagA, pilS, exc, traF, pilU, pilQ 
--> pilus biogenesis (pilU, pilQ, pilR, pilV, pilS), DNA modification/relaxosome/plasmid transfer (traY, yagA, exc, traF)


In [ ]:
merged[(merged["LFC_9_1"]<0) & (merged["LFC_10_1"]<0) & (merged["LFC_10_2"]<0) & (merged["LFC_11_2"]<0) & (merged["LFC_12_1"]<0) & (merged["LFC_12_2"]<0) & (merged["LFC_13_1"]<0) & (merged["LFC_13_2"]<0) & (merged["LFC_14_2"]<0) & (merged["LFC_15_1"]<0)]
# "LFC_9_1", "LFC_10_1", "LFC_10_2", "LFC_11_2", "LFC_12_1", "LFC_12_2", "LFC_13_1", "LFC_13_2", "LFC_14_2", "LFC_15_1"

#### Looking closer at the correlation between LFC10_1 and LFC_11_2 + LFC_12_1
Pilus genes and conjugation genes (tra) frequently appear here

LFC_10_1 and LFC_11_2 both negative LFC: transmembrane transport (ydiM, SL1344_1564, SL1344_1472, ompN, ssaV), metabolism gluconic acid (yafB), prgH (virulence), ssrA (stalled protein biosynthesis), hypothetical bacteriophage tail protein (SL1344_1929)

LFC_10_1 and LFC_11_2 both positive LFC: pathogenicity (sii genes, pipB2, SL1344_2880, spaS, hilA), bacteriophage tail (SL1344_2675), colonization (SL1344_0084), membrane (SL1344_4139, rfaB, SL1344_3000, SL1344_4052, SL1344_3799, rfaZ, SL1344_0338), metabolism (SL1344_3227, SL1344_3092, ttdB, SL1344_3106, iolE, SL1344_3958, SL1344_3107, assT, SL1344_3746), chemotaxis (SL1344_3112), pilus (stjB), transport (SL1344_3748, yihO, ompL, mgtB), transcription (leuO), cell invasion (sipC), protein assembly (lpfB, stjC), restriction system (SL1344_4424), peptidoglycan (oafA)



In [ ]:
merged[(merged["LFC_10_1"]<0) & (merged["LFC_11_2"]<0)][["Name", "number_of_barcodes_10_1", "LFC_10_1", "neg_selection_fdr_10_1", "pos_selection_fdr_10_1", "number_of_barcodes_11_2", "LFC_11_2", "neg_selection_fdr_11_2", "pos_selection_fdr_11_2"]]
#merged[(merged["LFC_10_1"]>0) & (merged["LFC_11_2"]>0)][["Name", "number_of_barcodes_10_1", "LFC_10_1", "neg_selection_fdr_10_1", "pos_selection_fdr_10_1", "number_of_barcodes_11_2", "LFC_11_2", "neg_selection_fdr_11_2", "pos_selection_fdr_11_2"]]


LFC_10_1 and LFC_12_1 both negative LFC: pilus genes, conjugation genes (tra), type 3 secretion system (invG)


LFC_10_1 and LFC_12_1 both positive LFC: virulence (pipB2, SL1344_2880, sipC, hilA), metabolism (SL1344_3227, SL1344_3092, SL1344_4047, ttdB, yjcC, SL1344_3747, yiaH, SL1344_3106, iolE, rfaZ, assT, SL1344_3934, SL1344_3746), transport (siiD, SL1344_3000, yhjV, SL1344_3748, ompL, siiF, mgtB, siiB), membrane (SL1344_4052, SL1344_3112, yihO), pilus (stjB), restriction system (SL1344_4424), protein folding (stjC), peptidoglycan (oafA)



In [ ]:
merged[(merged["LFC_10_1"]<0) & (merged["LFC_12_1"]<0)][["Name", "number_of_barcodes_10_1", "LFC_10_1", "neg_selection_fdr_10_1", "pos_selection_fdr_10_1", "number_of_barcodes_12_1", "LFC_12_1", "neg_selection_fdr_12_1", "pos_selection_fdr_12_1"]]
#merged[(merged["LFC_10_1"]>0) & (merged["LFC_12_1"]>0)][["Name", "number_of_barcodes_10_1", "LFC_10_1", "neg_selection_fdr_10_1", "pos_selection_fdr_10_1", "number_of_barcodes_12_1", "LFC_12_1", "neg_selection_fdr_12_1", "pos_selection_fdr_12_1"]]
